# (3) Logs Analysis using Mining. 
# Data set type: Industial-Anoki

# 💨🔥💨 Smoke Analysis

#### ✅ python, ✅ Gitlab, ✅ Mongodb

## Qs
### Q1: What is the top of most common problems in pipelines?

## Index

- [Import python libraries](#Import-python-libraries)
- [Page reference](#Page-reference)

## Import python libraries

In [195]:
import os
import pymongo
from pymongo import MongoClient

# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import string
import re
# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
# Plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
# ==============================================================================

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
# Configuración warnings
# ==============================================================================
import warnings

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ceciliocannavaciuolo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Config Variables

In [170]:
plotly_template="plotly_dark"
# plotly_template="plotly"

# Page reference

- [TextBlob: Librarian documentation to do text analysis](https://textblob.readthedodfdfdfio/en/dev/quicksdfadftdfhtml#get-word-and-noun-phrase-frequencies)
- pipenv: https://pipenv.pydfdfdfio/en/latest/
- pythonplot: https://pythonplot.com/

# Get data from MongoDB
### Read data from Mongodb database

In [354]:
# Connect with DB
MONGODB_URL = os.environ.get('MONGODB_URL')
NODE_ENV = os.environ.get('NODE_ENV') or "dev"
DB_NAME = os.environ.get('APP_NAME') + "-"+ NODE_ENV

client = MongoClient()
client = MongoClient(MONGODB_URL)
db = client[DB_NAME]

# jobs = db.gitlablogs.find({}).limit(500) # Read all data
jobs = db.gitlablogs.find({}) # Read all data

jobs = pd.DataFrame(list(jobs)) # Convert to DataFrame
print("List of data available iside of db structure")
jobs.dtypes

List of data available iside of db structure


_id                                 object
jobId                                int64
projectId                            int64
__v                                  int64
allow_failure                         bool
commitId                            object
commitMessage                       object
commitTitle                         object
committedEmail                      object
created_at                  datetime64[ns]
duration                           float64
jobName                             object
jobRef                              object
jobStage                            object
jobStatus                           object
pipelineId                           int64
pipelineRef                         object
pipelineStatus                      object
pipelineUrl                         object
pipelineWebUrl                      object
projectDescriptions                 object
projectName                         object
projectNameWithNamespace            object
queued_dura

# Analysis of data volumes.
## In this section you can obtain general information related to the volume of data.

## Percentage of type jobs

In [355]:
# jobs
print("------ DATA REPORT ------")
projectsNumber = len(jobs["projectName"].unique())
print("Number of Projects: "+ str(projectsNumber))
numberOfJobs = len(jobs.index)
print("Number of pipelines Jobs (Steps): "+ str(numberOfJobs))
numberOfSuccess= jobs.loc[jobs["jobStatus"] == "success"]["jobStatus"].count()
numberOfFailed= jobs.loc[jobs["jobStatus"] == "failed"]["jobStatus"].count()
numberOfCancel= jobs.loc[jobs["jobStatus"] == "canceled"]["jobStatus"].count()

successPercentage = (1-((numberOfSuccess+numberOfFailed+numberOfCancel)-numberOfSuccess)/(numberOfSuccess+numberOfFailed+numberOfCancel))*100
failedPercentage = (1-((numberOfSuccess+numberOfFailed+numberOfCancel)-numberOfFailed)/(numberOfSuccess+numberOfFailed+numberOfCancel))*100
canceledPercentage = (1-((numberOfSuccess+numberOfFailed+numberOfCancel)-numberOfCancel)/(numberOfSuccess+numberOfFailed+numberOfCancel))*100

print("Number of success Jobs (Steps): "+ str(numberOfSuccess) + " or "+str(successPercentage) + " %")
print("Number of failed Jobs (Steps): "+ str(numberOfFailed)+ " or "+str(failedPercentage) + " %")
print("Number of canceled Jobs (Steps): "+ str(numberOfCancel)+ " or "+str(canceledPercentage) + " %")

------ DATA REPORT ------
Number of Projects: 8
Number of pipelines Jobs (Steps): 7159
Number of success Jobs (Steps): 5307 or 79.94877975293764 %
Number of failed Jobs (Steps): 375 or 5.649291955408254 %
Number of canceled Jobs (Steps): 956 or 14.401928291654109 %


In [356]:
fig = make_subplots(rows=1, cols=2)
fig = px.pie(jobs, names='jobStatus', title='Pipelines Jobs results',color="jobStatus",template=plotly_template)
fig.show()

['success', 'failed', 'canceled', 'skipped']

# Number of fails by stage number. 

In [381]:
def createBarGraphByJobStatus(variable):
    jobStatusUnique = jobs["jobStatus"].unique().tolist()
    df_list = []
    for status in jobStatusUnique:
        jobs_total = jobs.rename(columns={'jobStatus': status})
        total = jobs_total.groupby(by=variable)[status].count()
        df_list.append(total)

    df = pd.concat(df_list,axis=1)
    fig = px.bar(df, orientation='h',template=plotly_template,title="Number of fails by "+ variable)
    fig.show()

print(" Number of jobs stage types")
createBarGraphByJobStatus("jobStage")
print(" Number of jobs projectName types")
createBarGraphByJobStatus("projectName")

 Number of jobs stage types


 Number of jobs projectName types


## Number of fails by job state

# Filter logs data


## Get fragment of text with error 

In [11]:
# Get fragment of text with error
# ==============================================================================
def getErrorText(texto):
    #! Get only last range. 
    nuevo_texto = texto
    #! Convert all text to lowercase.
    nuevo_texto = nuevo_texto.lower()
    #! Web page removal (words beginning with "http")
    nuevo_texto = re.sub('http\S+', ' ', nuevo_texto)
    nuevo_texto = nuevo_texto.split(sep = '\n')
    
    whiteList = ["error"]
    newTextList = []
    nuevoTexto = ""

    for text in nuevo_texto:
        for listI in whiteList:
            if listI in text:
                #! Remove special characters
                regex = '[\\!\\"\\#\\>\\<\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\;\\\\\]\\<\\=\\>\\?\\:\\@\\@\\\\^_\\`\\{\\|\\}\\~]'
                # text = re.sub(regex , '', text)
                text = re.sub('http\S+', ' ', text)
                #! Remove numbers
                text = re.sub('\d+', ' ', text)
                text = re.sub('- - t : :', '', text)
                # text = re.sub('- - t : dfdfdf z time=" - - t df df df z"', ' ', text)
                #! remove date
                text = re.sub('\d{4}-\d{2}-\d{2}', ' ', text)

                # text = re.sub('hosttcp/dfddfddff dff df', ' ', text)

                if not text in newTextList:
                    #! cut string. 
                    text = text[0:text.find(".")]
                    nuevoTexto = nuevoTexto + "\n" + text
                    newTextList.append(text)

    # nuevoTexto  = nuevoTexto.split(sep = ' ')
    return newTextList

## Test Filter for search the text error inside of the logs

In [12]:
textExample = '''
 * [new branch]      frontend-test           -> origin/frontend-test
 * [new branch]      functional-testing-junit -> origin/functional-testing-junit
 * [new branch]      master                  -> origin/master
 * [new branch]      sol-dev                 -> origin/sol-dev
 * [new branch]      sql-fix-branch          -> origin/sql-fix-branch
 * [new branch]      testing-report          -> origin/testing-report
 * [new tag]         CR-H1-2021-deploy-intermedio -> CR-H1-2021-deploy-intermedio
 * [new tag]         v1dffdfdfdf0_20190405         -df dff1dff0df0_20190405
 * [new tag]         v1dffdfdfdf1_20190424         -df dff1dff0df1_20190424
Checking out 18b927c2 as devdfdfdffdffdf

Skipping Git submodules setup
Downloading artifacts for build-jar (1473723797)dfdfdffdffdf
Downloading artifacts from coordinatordfdfdffdffdf ok        id=1473723797 responseStatus=200 OK token=G4kf2uft
$ echo "{\"auths\":{\"$CI_REGISTRY\":{\"username\":\"$CI_REGISTRY_USER\",\"password\":\"$CI_REGISTRY_PASSWORD\"}}}" > /kaniko/.docker/confdfdfdfjson
$ /kaniko/executor --context $CI_PROJECT_DIR --dockerfile $CI_PROJECT_DIR/Dockerfile --destination $CI_REGISTRY_IMAGE:$CI_COMMIT_REF_SLUG
INFO[0001] Retrieving image manifest openjdk:11.0       
INFO[0001] Retrieving image openjdk:11.0 from registry inddfdfdfdodfkdfrdfio 
error building image: GET https://index.dockdfdfdfio/v2/library/openjdk/manifestdf/df1df0: TOOMANYREQUESTS: You have reached your pull ratedflidfitdf You may increase the limit by authenticating and upgrading: httdfs:/dfwdfwdfddfckerdfcom/increase-rate-limit
ERROR: Job failed: command terminated with exit code 1
'''

errorText =  getErrorText(textExample)

print(errorText)

# This need to do defferent to 0.  
assert  len(errorText) > 0
print("✅ PASS TEST: OK 👍")

textExample = '''
    >>>> 03_add_account_toEntity-06 :              |'\n  │ '|------------------------------------------------|'\n  │ \n  │ 'https://edutelling-api-develop.openshidfdfdftecdfgdfpdfit/a\n  │ pi/v1/tutors/create/tutor-from-ambassador?ambassadorId\n  │ ='\n  │ 'RESPONSE'\n  │ \n  │ `{\"success\":true,\"message\":\"Tutor '#17:-2' cdfeadfeddf\",\"\n  │ data\":{\"tutorId\":\"#17:-2\"}}`\n  │ '|-**********************************************-|'\n  │ \n  └\n\n→ 04_login_no2_v3\n  POST https://edutelling-apidfdevdflopddfopendfhdfftdftdfchgapdfit/api/v1/auth/authentication [200 OK, 798B, 137ms]\n  ✓  [(POST)/api/v1/auth/authentication] Login Delete (200)\n\n→ 05_login_switch_how_account\n  POST https://edutellidfg-api-dfedfelopdfodffenshiftdfftechgapdfit/api/v1/auth/complete-authedftication df200 OK, 1df01KB, 27ms]\n  ✓  CHECK IF EXIST JWT\n  ✓  CHECK IF EXIST jwtRefresh\n  ✓  [(POST)/api/v1/auth/authentication] Login Delete (200)\n\n→ 01_entityTypeAccountService_getAll\n  GET https://dfdutellingdfdfpi-dedfelodfdfopensdfiftdftechgapdfit/apdf/v1/accounts/dfmbassador@botdfcom/all [200 OK, 680B, 23ms]\n  ┌\n  │ 'deleteAccountId :', '#12:1564'\n  │ 'deleteAccountId :', '12%3A1564'\n  └\n\n→ 02_reamoveAccount\n  DELETE dfttps://eddftelldfndf-api-dedfelopdfodfenshiftdftechgapdfit/api/v1/accounts/remove?dataId=12%3A1564 [200 OK, 397B, 15ms]\n\n→ goTo_workflowControl\n  OPdfIONS httpdf://edutdfflling-apidfdevelopdffopenshiftdftechgapdfit [404 Not Found, 255B, 4ms]\n\nAttempting to set next request to controlOfWorkFlowTempalte_06\n\n→ controlOfWorkFlowTempalte_06dfn  OPTIONdf https:df/edfutelling-dfpi-devedfopdfopenshiftdftechgapdfit [404 Not Found, 255B, 6ms]\n  ┌\n  │ ' ------ ------ ---------- ---------'\n  │ \n  │ '{\"nameOfConfigFile\":\"config_multiAccount_tutorHowAmba\n  │ ssador\",\"templateRun\":[\"01_entityTypeAccountService_ge\n  │ tAll_06\",\"config_multiAccount_studentHowAmbassador\"],\"\n  │ executed\":[false,true]}'\n  │ \n  │ 'WORKFLOW GO TO: ------------------------'\n  │ \n  │ 'WORKFLOW CONTROL REDIRECTING TEMPATE TO: ', 'config_m\n  │ ultiAccount_studentHowAmbassador'\n  │ \n  │ \n  └\n\nAttempting to set next request to config_multiAccount_studentHowAmbassador\n\n→ config_multiAccount_studentHowAmbadfsador\n  dfPTIONS dfttps:df/edutellidfg-api-ddfvelopdfopenshiftdftechgapdfit [404 Not Found, 255B, 5ms]\n\nAttempting to set next request to controlOfWorkFlowTempalte_06\n\n→ controlOfWorkFlodfTempalte_df6\n  OPdfIONS httdfs://edutedfling-apdf-developdfopenshiftdftechgapdfit [404 Not Found, 255B, 8ms]\n  ┌\n  │ ' ------ ------ ---------- ---------'\n  │ \n  │ '{\"nameOfConfigFile\":\"config_multiAccount_tutorHowAmba\n  │ ssador\",\"templateRun\":[\"01_entityTypeAccountService_ge\n  │ tAll_06\",\"stop_06_createCourseModuleAndStage\"]}'\n  │ \n  │ 'WORKFLOW GO TO: ------------------------'\n  │ \n  │ 'WORKFLOW CONTROL REDIRECTING TEMPATE TO: ', '01_entit\n  │ yTypeAccountService_getAll_06'\n  │ \n  │ \n  └\n\nAttempting to set next request to 01_entityTypeAccountService_getAll_06\n\n→ 01_entityTypeAcdfountServidfe_getAldf_06\n  GET dfttps://eddftellindfdfapi-developdfopenshiftdftechgadfdfit/api/v1/accounts/student@botdfcom/all [200 OK, 475B, 8ms]\n  ┌\n  │ 'entityTypeUppperCase: ', 'Student'\n  │ '20%3A273'\n  df\n\n→ 02_dfeamoveAdfcount_06\n  DELdfTE https:df/eduteldfing-api-developdfopenshiftdftechgapdfit/api/v1/accounts/remove?dataId=12%3A1564 [200 OK, 379B, 53dfs]\n\n→ 0df_add_acdfount_toEntity-06\ndf POST httdfs://edudfelling-api-developdfopenshiftdftechgapdfit/api/v1/ambassadors/create/ambassador-from-student?studentId=20%3A273 [200 OK, 420B, 84ms]\n  ✓   [(POST) /api/v1/teachers/create/teacher-from-student?studentId ] Status code is 200\n  ✓  Check if was Success to add access student how teacher\n  ┌\n  │ \n  │ '|------------------------------------------------|'\n  │ '| >>>> 03_add_account_toEntity-06 :              |'\n  │ '|-------------df---------df-------df----------------|'\n df│ \n  │ 'dfttps://dfdutelling-api-developdfopenshiftdftechgapdfit/a\n  │ pi/v1/ambassadors/create/ambassador-from-student?stude\n  │ ntId='\n  │ 'RdfSPONSE'\n  │ \n  │ `{\"success\":true,\"medfsage\":\"Ambassador '#19:-2' create\n  │ ddf\",\"data\":{\"ambassadorId\":\"#19:-2\"}}`\n  │ '|-********************************df*********df***-|'\df  │ \n  └\n\n→ 04_login_ndf2_v3\n  PdfST httpdf://edutelling-api-developdfopenshiftdftechgapdfit/api/v1/auth/authentication [200 OK, 790B, 152ms]\n  ✓  [(POST)/api/v1/auth/authenticatidfn] Login dfelete (df00)\n\n→ 05_login_switch_howdfaccount\ndf POST hddftps://edutelling-api-developdfopenshiftdftechgapddfit/api/v1/auth/complete-authentication [200 OK, 1df03KB, 23ms]\n  ✓  CHECK IF EXIST JWT\n  ✓  CHECK IF EXIST jwtRefresh\n  ✓  [(POST)/api/v1/auth/authenticadfion] Logidf Deletedf(200)\n\n→ 01_entityTypeAccoundfSdfrvice_getdfll\n  GdfT https://edutelling-api-develdfpdfopenshiftdftechgapdfit/api/v1/accounts/student@botdfcom/all [200 OK, 676B, 9ms]\n  ┌\n  │ 'deleteAccountId :', '#12:15df5'\n  │ 'dfeleteAcdfountId :', '12%3A1565'\n  └\n\n→ 02_dfeamoveAccdfunt\n  dfELETE https://edutelling-api-developdfopenshiftdftechgapdfit/api/v1/accoudfts/removedfdataId=df2%3A1565 [200 OK, 397B, 37ms]\n\n→ goTodfworkflowCdfntrol\ndf OPTIONS https://edutelling-api-developdfopenshiftdftechgapdfit [404 Not Found, 255B, 6ms]\n\nAttempting to sedf next reqdfest to dfontrolOfWorkFlowTempalte_06\n\n→ controlOfdforkFlowTedfpalte_0df\n  OPTIONS https://edutelling-api-developdfopenshiftdftechgapdfit [404 Not Found, 255B, 4ms]\n  ┌\n  │ ' ------ ------ ---------- ---------'\n  │ \n  │ '{\"nameOfConfigFile\":\"config_multiAccount_tutorHowAmba\n  │ ssador\",\"templateRun\":[\"01_entityTypeAccountService_ge\n  │ tAll_06\",\"stop_06_createCourseModuleAndStage\"],\"execut\n  │ ed\":[false,true]}'\n  │ \n  │ 'WORKFLOW GO TO: ------------------------'\n  │ \n  │ 'WORKFLOW CONTROL REDIRECTING TEMPATE TO: ', 'stop_06_\n  │ createCourseModuleAndStage'\n  │ \n  │ \n  └\n\nAttempting to set nextdfrequest tdf stop_0df_createCourseModuleAndStage\n\n→ stop_06_creadfeCourseModfuleAndSdfage\n  OPTIONS httpdf://edutelling-api-developdfopenshiftdftechgapdfit [404 Not Found, df55B, 5ms]\nsummary: 0\nNUMBER OF FAILS 0\ncollection run completeddf\n\n┌─────────────────────────┬───────────────────┬──────────────────┐\n│                         │          executed │           failed │\n├─────────────────────────┼───────────────────┼──────────────────┤\n│              iterations │                 1 │                0 │\n├─────────────────────────┼───────────────────┼──────────────────┤\n│                requests │               425 │                0 │\n├─────────────────────────┼───────────────────┼──────────────────┤\n│            test-scripts │               425 │                0 │\n├─────────────────────────┼───────────────────┼──────────────────┤\n│      prerequest-scripts │                 3 │                0 │\n├─────────────────────────┼───────────────────┼──────────────────┤\n│              assertions │df              247 │                0 │\n├─────────────────────────┴df──────────────────┴──────────────────┤\n│ total run duration: 4m df9df3s                                   │\n├────────────────────────df───────────────────────────────────────┤\n│ total data received: 755df83KB (approx)          df df            │\n├───────────────────────────────────────────────────df─df──────────┤\n│ avedfagedfresponse time: 44ms [min: 3ms, max: 605ms, sdfddf: 58ms] │\n└──df───df───ddf───────df─────────────────────────────────────────────┘\nDone indfdf60df84sdddf\n\u001dff[32;1m$ echo 'http://edutelling-functional-test-backenddfdfpenshiftddftechgapdffit/'\u001b[0;m\nhttp://edutelling-functional-test-backenddfopenshiftdftechgapdfit/\nsection_end:1626336539:step_script\r\u001b[0Ksection_start:1626336539:cleanup_file_variables\r\u001b[0K\u001b[0K\u001b[36;1mCleaning up file based variables\u001b[0;m\n\u001b[0;msection_end:1626336540:cleanup_file_variables\r\u001b[0K\u001b[32;1mJob succeeded\n\u001b[0;m
'''
errorText =  getErrorText(textExample)
print(errorText)
assert  len(errorText) == 0
print("✅ PASS TEST: OK 👍")

['error building image: get   toomanyrequests: you have reached your pull ratedflidfitdf you may increase the limit by authenticating and upgrading: httdfs:/dfwdfwdfddfckerdfcom/increase-rate-limi', 'error: job failed: command terminated with exit code ']
✅ PASS TEST: OK 👍
[]
✅ PASS TEST: OK 👍


## Apply filter to all data 

In [13]:
# The cleaning and tokenization function is applied to each job
# ==============================================================================
df = jobs
df['jobLog_token'] = df['jobLog'].apply(lambda x: getErrorText(x))
df[['jobLog', 'jobLog_token']].head(10)

,jobLog,jobLog_token
0,[0KRunning with gitlab-runner 14.1.0-rc1 (e94...,"[health check error, , , , , , , ..."
1,[0KRunning with gitlab-runner 14.1.0-rc1 (e94...,"[health check error, , , , , ]"
2,[0KRunning with gitlab-runner 14.1.0-rc1 (e94...,"[health check error, , , , , , , ..."
3,[0KRunning with gitlab-runner 14.1.0-rc1 (e94...,"[health check error, , , , , , ]"
4,[0KRunning with gitlab-runner 14.1.0-rc1 (e94...,"[health check error, , , , , , ]"
5,[0KRunning with gitlab-runner 14.1.0-rc1 (e94...,"[health check error, , , , , ]"
6,[0KRunning with gitlab-runner 14.1.0-rc1 (e94...,"[health check error, , , , , , , ..."
7,[0KRunning with gitlab-runner 14.1.0-rc1 (e94...,"[health check error, , , , , , ]"
8,[0KRunning with gitlab-runner 14.1.0-rc1 (e94...,"[health check error, , , , , , , ..."
9,[0KRunning with gitlab-runner 14.1.0-rc1 (e94...,"[health check error, , , , , ]"


# Remove stopwords


In [24]:
# Obtaining a list of stopwords in English
# ==============================================================================
stop_words = list(stopwords.words('english'))
# Se añade la stoprword: amp, ax, ex
stop_words.extend(("amp", "xa", "xe"))
print(stop_words[:10])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


# Exploratory analysis

### In Python, one of the structures that most facilitates exploratory analysis is the Pandas DataFrame, which is the structure in which the information from the df is now stored. However, when tokenizing, there has been a major chandfdfdf Before dividing the text, the study elements were the df, and each one was in a row, thus fulfilling the condition of tidy data: an observation, adfrdfwdf When performing the tokenization, the element of study has become each token (word), thus violating the condition of tiddf ddftadf To get back to the ideal structure, each token list has to be expanded, doubling the value of the other columns as many times as ndfcesdfarydf This process is known as expansiondfor udfnestdf


### Although it may seem an inefficient process (the number of rows increases a lot), this simple change facilitates activities of the type: grouping, counting, graphics dfdfdffdffdf


In [25]:
# Unnest de la columna texto_tokenizado
# ==============================================================================
jobs_tidy = df.explode(column='jobLog_token')
jobs_tidy = jobs_tidy.drop(columns='jobLog')
jobs_tidy = jobs_tidy.rename(columns={'jobLog_token':'token'})
jobs_tidy.head(3)


,_id,jobId,projectId,__v,allow_failure,commitId,commitMessage,commitTitle,committedEmail,created_at,...,projectNameWithNamespace,queued_duration,runnerDescription,runnerId,runnerIpAddress,runnerName,sha,started_at,username,token
0,611102da01a14071451c22c7,1452810390,15112024,0,False,fa3cb44c5c76f4955d7b63f6159bc17916c4a29b,Merge branch '226-A-getStagesByTeacherId' into...,Merge branch '226-A-getStagesByTeacherId' into...,c.cannavacciuolo@anoki.it,2021-07-26 15:10:20.281,...,Anoki S.r.l. / EDUtelling-Prj / edutelling-api,0.257544,shared-runners-manager-7,None,35.237.21.158,gitlab-runner,fa3cb44c5c76f4955d7b63f6159bc17916c4a29b,2021-07-26 15:23:34.058,cecilio.cannav,health check error
0,611102da01a14071451c22c7,1452810390,15112024,0,False,fa3cb44c5c76f4955d7b63f6159bc17916c4a29b,Merge branch '226-A-getStagesByTeacherId' into...,Merge branch '226-A-getStagesByTeacherId' into...,c.cannavacciuolo@anoki.it,2021-07-26 15:10:20.281,...,Anoki S.r.l. / EDUtelling-Prj / edutelling-api,0.257544,shared-runners-manager-7,None,35.237.21.158,gitlab-runner,fa3cb44c5c76f4955d7b63f6159bc17916c4a29b,2021-07-26 15:23:34.058,cecilio.cannav,
0,611102da01a14071451c22c7,1452810390,15112024,0,False,fa3cb44c5c76f4955d7b63f6159bc17916c4a29b,Merge branch '226-A-getStagesByTeacherId' into...,Merge branch '226-A-getStagesByTeacherId' into...,c.cannavacciuolo@anoki.it,2021-07-26 15:10:20.281,...,Anoki S.r.l. / EDUtelling-Prj / edutelling-api,0.257544,shared-runners-manager-7,None,35.237.21.158,gitlab-runner,fa3cb44c5c76f4955d7b63f6159bc17916c4a29b,2021-07-26 15:23:34.058,cecilio.cannav,


# Total words used by each log event

In [16]:
jobs_tidy.groupby(by='jobStatus')['token'].count()

jobStatus
canceled      95
failed       299
skipped        0
success     1883
Name: token, dtype: int64

# Total words used by each project

In [17]:
jobs_tidy.groupby(by='projectName')['token'].count()

projectName
API                     174
EduTelling              345
TRusT-FE                 86
app-prenotazioni-api    775
ckp-api                 276
edutelling-api          609
pax-Italia-pot-cicd      12
trust-mail-sender         0
Name: token, dtype: int64

# Frequency of words

In [18]:
# Total words per event
# ==============================================================================
print('--------------------------')
print('Total words per event')
print('--------------------------')
jobs_tidy.groupby(by='jobStatus')['token'].nunique()

--------------------------
Total words per event
--------------------------


jobStatus
canceled     5
failed      48
skipped      0
success     29
Name: token, dtype: int64

In [19]:
# Longitud media y desviación de los jobs de cada jobStatus
# ==============================================================================
temp_df = pd.DataFrame(jobs_tidy.groupby(by = ["jobStatus", "jobId"])["token"].count())
temp_df.reset_index().groupby("jobStatus")["token"].agg(['mean', 'std'])

,mean,std
jobStatus,,
canceled,0.826087,2.233167
failed,8.081081,6.738688
skipped,0.000000,0.000000
success,5.793846,3.516797


In [20]:
# Top 50 palabras más utilizadas por cada evento
# ==============================================================================
jobs_tidy.groupby(['jobStatus','token'])['token'] \
 .count() \
 .reset_index(name='count') \
 .groupby('jobStatus') \
 .apply(lambda x: x.sort_values('count', ascending=False).head(10))
 

jobStatus                                              token  \
jobStatus                                                                   
canceled  0   canceled                                                      
          2   canceled                                 health check error   
          1   canceled             [info] error stacktraces are turned on   
          3   canceled          warning: illegal reflective access by com   
          4   canceled  warning: please consider reporting this to the...   
failed    8     failed                                                      
          5     failed  [ ;msection_end: :cleanup_file_variables\r[ ...   
          50    failed                                 health check error   
          15    failed      error: cannot find module '@vue/cli-plugin...   
          32    failed                                🛑  error: smoke tes   
          33    failed                               🛑  smoke test error    
          51    failed  section_end: :build_script\r[ ksection_start:...   
          38    failed  error building image: get   toomanyrequests: y...   
          14    failed      + fullyqualifiederrorid : commandnotfounde...   
          6     failed  [ ;msection_end: :cleanup_file_variables\r[ ...   
success   56   success                                                      
          78   success                                 health check error   
          59   success             [info] error stacktraces are turned on   
          67   success                                       <s> [webpack   
          80   success          warning: illegal reflective access by com   
          81   success  warning: please consider reporting this to the...   
          54   success  [ ; merror: downloading artifacts from coordi...   
          55   success                            [ ; mwarning: retrying   
          79   success        module not found: error: can't resolve 'hls   
          61   success  (emitted value instead of an instance of error...   

              count  
jobStatus            
canceled  0      78  
          2      13  
          1       2  
          3       1  
          4       1  
failed    8     119  
          5      20  
          50     20  
          15     18  
          32     16  
          33     16  
          51      8  
          38      7  
          14      6  
          6       6  
success   56   1347  
          78    215  
          59     57  
          67     55  
          80     35  
          81     35  
          54     22  
          55     22  
          79     16  
          61     13

In [21]:
# # Top 10 palabras por dftatus (sin stopwords)
# # ==============================================================================
# fig, axs = plt.subplots(nrows=3, ncols=1,figsize=(50, 50))
# for i, dftatus in enumerate(df_tidy.dftatdfdfdfunique()):

#     try:
#         df_temp = df_tidy[df_tidy.dftatus == dftatus]
#         counts  = df_temp['token'].value_counts(ascending=Falsdfdfdfhead(10)
#         counts.plot(kind='barh', color='firebrick', ax=axs[i])
#         axs[i].invert_yaxis()
#         axs[i].set_title(dftatus)
#     except:
#         print("Error")

# fig.tight_layout()


In [22]:
# import plotly.express as px
# df = px.dadfdfdfgapminder()
# fig = px.scatter(df, x="gdpPercap", y="lifeExp", animation_frame="year", animation_group="country",
# size="pop", color="continent", hover_name="country",
# log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90])

# fig["layout"].pop("updatemenus") # optional, drop animation buttons

# fig.show(renderer="notebook")



In [23]:
# import plotly.express as px
# df = px.dadfdfdftips()
# fig = px.bar(df, x="sex", y="total_bill", color="smoker", barmode="group")
# fig.show()